In [1]:
from copy import deepcopy
from itertools import combinations
import pickle
import typing as tp
from zipfile import ZipFile
import pickle
from lightfm import LightFM
from lightfm.data import Dataset as LFMDataset
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.preprocessing import normalize
from transliterate import translit
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

C:\ProgramData\Anaconda3\envs\torchvision2\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
RANDOM_STATE = 42

## Загрузка данных

Для целей семинара используем анонимизированные данные по просмотрам в онлайн-кинотеатре Кион. Мы имеем данные по действиям примерно за полгода, а также признаки пользователей и контента.

In [4]:
interactions = pd.read_csv(r'../../data/recsys-in-practice/train_joke_df.csv')
interactions

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60
...,...,...,...
1448359,22604,26,2.82
1448360,22255,36,-1.94
1448361,21056,40,-9.56
1448362,12328,97,0.87


In [5]:
rating = interactions['Rating'].values
print(np.min(rating), np.max(np.abs(rating)), np.max(np.abs(rating)))

rating_norm = (rating - np.min(rating)) / (np.max(rating) - np.min(rating))
#rating_norm = rating / np.max(np.abs(rating))
print(np.min(rating_norm), np.max(rating_norm))

interactions['Rating_norm'] = rating_norm

-9.95 10.0 10.0
0.0 1.0


In [6]:
interactions.head()

,UID,JID,Rating,Rating_norm
0,18029,6,-1.26,0.435589
1,3298,64,-4.17,0.289724
2,3366,58,0.92,0.544862
3,12735,92,3.69,0.683709
4,11365,38,-6.60,0.167920


Определим последний доступный день в выборке как тестовый, а 3 месяца до него - обучающей выборкой.

Для простоты:
- удачным взаимодействием будем считать просмотр любой длительности и при обучении будем использовать только эту информацию, игнорируя данные о признаках пользователей и контента
- не будем обрабатывать "холодных" пользователей, просто удалим их из тестовой выборки

In [7]:
train, test = train_test_split(interactions, test_size=0.15, random_state=42)

In [6]:
#assert False

## Добавление аватаров в обучающую выборку

Далее для построения рекомендаций будем использовать алгоритм матричного разложения из библиотеки LightFM. Для формирования рекомендаций для аватаров необходимо, чтобы действия аватаров были в обучающей выборке.

Отметим, что добавляя действия аватаров в обучающую выборку, мы несколько изменяем ее распределение. Обычно этим можно пренебречь, т.к. объем обучающей выборки (в нашем случае 5 млн наблюдений) значительно больше объема действий аватаров (10 наблюдений).

## Обучение модели

In [5]:
lfm_dataset = LFMDataset()
lfm_dataset.fit(
    users=train["UID"].values,
    items=train["JID"].values,
)

train_matrix, _ = lfm_dataset.build_interactions(zip(*train[["UID", "JID"]].values.T))

In [8]:
lfm_model = LightFM(
    learning_rate=0.01, 
    loss='warp', 
    no_components=64,
    random_state=RANDOM_STATE
)
lfm_model.fit(
    interactions=train_matrix, 
    epochs=15,
    num_threads=20
);

In [9]:
with open('jokes_lfm_model.pkl', 'wb') as f:
    pickle.dump(lfm_model, f)

In [10]:
assert False

AssertionError: 

## Рекомендации для аватаров

In [ ]:
n_recommendations = 10

In [ ]:
id_item_mapping = {v: k for k, v in lfm_dataset._item_id_mapping.items()}

In [ ]:
def get_n_recommendations_for_user(
    user_id: str,
    model: LightFM,
    train_matrix: coo_matrix,
    user_to_id: tp.Dict[str, int],
    id_to_item: tp.Dict[int, str],
    n_recommendations: int,
    model_name: str
) -> pd.DataFrame:
    
    if model_name == 'random':
        return np.random.choice(train["JID"].values, n_recommendations)
    
    if model_name == 'popular':
        return list(train["JID"].value_counts()[:n_recommendations].index)
    
    user_inner_id = user_to_id[user_id]
    scores = model.predict(
        user_ids=user_inner_id,
        item_ids=np.arange(train_matrix.shape[1]),
        num_threads=20
    )
    user_watched_items = train_matrix.col[train_matrix.row == user_inner_id]
    scores[user_watched_items] = -np.inf

    recommended_item_inner_ids = np.argpartition(scores, -np.arange(n_recommendations))[
        -n_recommendations:
    ][::-1]
    recommended_item_ids = [id_to_item[x] for x in recommended_item_inner_ids]
    return recommended_item_ids


In [ ]:
get_n_recommendations_for_user(
    user_id=1,
    model=lfm_model,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations,
    model_name=None
)

In [ ]:
user_id = 1

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations,
    model_name=None
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items})

In [ ]:
user_id = 2

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations,
    model_name=None
)
pd.DataFrame({"user_id": user_id, "item_id": recommended_items})

In [ ]:
# самые просматриваемые в обучающей выборке
train["JID"].value_counts().head(10)

Мы получили, что рекомендации для аватаров имеют сильное пересечение, обусловленное перекосом к рекомендациям популярного контента.

In [ ]:
assert False

In [ ]:
with open('jokes_lfm_model.pkl', 'rb') as f:
    lfm_model = pickle.load(f)

##  Попробуем побороться с перекосом к популярным

Алгоритм обучает для каждого пользователя $u$ и товара $i$ соответственно смещения $b_u$, $b_i$ и эмбеддинги $p_u$, $q_i$. Для формирования рекомендаций для пользователя выбираются товары, имеющие наибольшие значения скоров, определяющихся по формуле:

$$score_{ui} = b_u + b_i + p_u \cdot q_i = b_u + b_i + \cos ( p_u, q_i ) \cdot || p_u || \cdot || q_i || .$$

Часто перекос к популярным выражается в больших значениях смещений или норм эмбеддингов у популярных товаров. В таких случаях может помочь переход от ранжирования по значениям скалярных произведений к ранжированию по косинусам угла между эмбеддингами пользователей и товаров.

Для перехода к косинусам согласно формуле выше достаточно заменить $b_u$ и $b_i$ нулями и привести нормы $p_u$ и $q_i$ к единицам.

In [ ]:
lfm_model_cos = deepcopy(lfm_model)

lfm_model_cos.item_biases = np.zeros_like(lfm_model_cos.item_biases)
lfm_model_cos.user_biases = np.zeros_like(lfm_model_cos.user_biases)

lfm_model_cos.item_embeddings = normalize(lfm_model_cos.item_embeddings)
lfm_model_cos.user_embeddings = normalize(lfm_model_cos.user_embeddings)

In [ ]:
user_id = 1

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model_cos,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations,
    model_name=None
)
pd.DataFrame({"UID": user_id, "JID": recommended_items})

In [ ]:
user_id = 2

recommended_items = get_n_recommendations_for_user(
    user_id=user_id,
    model=lfm_model_cos,
    train_matrix=train_matrix,
    user_to_id=lfm_dataset._user_id_mapping,
    id_to_item=id_item_mapping,
    n_recommendations=n_recommendations,
    model_name=None
)
pd.DataFrame({"UID": user_id, "JID": recommended_items})

Видно, что рекомендации стали более персонализированными для аватаров и исчезло преобладание популярного контента. Однако говорить о том, что новая версия модели лучше рано. Выводы стоит делать после того, как будут в том числе проведены количественные оценки качества (оффлайн и онлайн метрики).

## Расчет рекомендаций

Сформируем таблицы рекомендаций для пользователей из тестовой выборки.

Далее с помощью этих таблиц будут оценены метрики beyond accuracy. Методологически вернее оценивать данные метрики на всей пользовательской базе, а не только для пользователей, имеющих действия в тестовой выборке. Здесь мы этим пренебрежем для экономии вычислительных ресурсов.

In [ ]:
models_dict = {"lfm": lfm_model, "lfm_cos": lfm_model_cos, 'random':None, 'popular':None}

In [ ]:
recommendations_dict = {}
for model_name, model in tqdm(models_dict.items()):
    recommendations = pd.DataFrame({"UID": test["UID"].unique()})
    recommendations["JID"] = recommendations["UID"].apply(
        get_n_recommendations_for_user,
        args=(
            model,
            train_matrix,
            lfm_dataset._user_id_mapping,
            id_item_mapping,
            n_recommendations,
            model_name
        ),
    )
    recommendations = recommendations.explode("JID")
    recommendations["rank"] = recommendations.groupby(["UID"]).cumcount() + 1
    recommendations_dict[model_name] = recommendations
    

In [ ]:
with open('jokes_recommendations_dict.pkl', 'wb') as f:
    pickle.dump(recommendations_dict, f)

In [ ]:
assert False

In [ ]:
with open('jokes_recommendations_dict.pkl', 'rb') as f:
    recommendations_dict = pickle.load(f)

# Оценка beyond accuracy метрик
## Intra-List Diversity

Оценим для обеих моделей среднее разнообразие контента в полках с помощью метрики $ILD$:

$$ ILD = \frac{1}{|R| ( |R| - 1 )} \sum_{i \in R} \sum_{j \in R} d(i, j) . $$

В качестве расстояния $d(i, j)$ используем [расстояние Хэмминга](https://neerc.ifmo.ru/wiki/index.php?title=Расстояние_Хэмминга) между one-hot векторами жанров. Пример расчета:

##### <center>d(10<font color='blue'>1</font>1<font color='blue'>1</font>01, 10<font color='red'>0</font>1<font color='red'>0</font>01) = 2.</center>



Для оценки расстояния Хэмминга между фильмами вытянем список жанров в one-hot векторы.

Модель, тяготеющая к рекомендации популярного контента, отдает более разнообразные полки по набору жанров. Это логично, т.к. подборки популярного контента обычно разнообразны по значениям различных наборов признаков.

## Mean Inverse User Frequency

Оценим новизну рекомендаций. Новизна товара обратно пропорциональна количеству пользователей, которые с ним взаимодействовали в обучающей выборке. Значение метрики для полки определяется как средняя "новизна" товаров в полке.

$$ MIUF = -\frac{1}{|R|} \sum_{i \in R} \log_2 \frac{|U_i|}{|U|} $$

In [ ]:
def calculate_mean_inv_user_frequency_per_user(recommendations: pd.DataFrame, train: pd.DataFrame) -> pd.Series:
    n_users = train["UID"].nunique()
    n_users_per_item = train.groupby("JID")["UID"].nunique()
    
    recommendations_ = recommendations[["UID", "JID"]].copy()
    recommendations_["n_users_per_item"] = recommendations_["JID"].map(n_users_per_item)
    recommendations_["inv_user_freq"] = -np.log2(recommendations_["n_users_per_item"] / n_users)
    return recommendations_[["UID", "inv_user_freq"]].groupby("UID").agg("mean")

In [ ]:
for model_name, recommendations in recommendations_dict.items():
    miuf_per_user = calculate_mean_inv_user_frequency_per_user(recommendations, train)
    print(f"model: {model_name}, mean miuf: {round(float(miuf_per_user.mean()), 2)}\n")

Естественно, модель, отдающая полки с популярным контентом, имеет значительно меньшее значение метрики, т.к. метрика принимает высокие значения в тех случаях, когда подборка состоит из контента в "длинном" хвосте.

## Serendipity

Оценим "сюрпризность" рекомендаций, т.е. комбинацию оценки новизны и релевантности контента для пользователя.

$$ Serendipity = \frac{1}{|R|} \sum_{i \in R} \max \left( P_i - P^{U}_i , 0 \right) \cdot rel_i $$

$$ P_i = \frac{|I| + 1 - rank_i}{|I|}; \, P^{U}_i = \frac{|I| + 1 - rank^{U}_i}{|I|} $$

In [ ]:
def get_value_popularity_ranks(values: pd.Series) -> pd.Series:
    value_counts = values.value_counts()
    counts_unique = value_counts.unique()
    count_rank_mapping = pd.Series(index=counts_unique, data=np.arange(len(counts_unique)) + 1)
    return value_counts.map(count_rank_mapping)


def calculate_serendipity_per_user(
    recommendations: pd.DataFrame,
    train: pd.DataFrame,
    test: pd.DataFrame,
) -> pd.Series:
    recommendations_ = pd.merge(recommendations, test[["UID", "JID"]], how="left", indicator=True)
    recommendations_["is_rel"] = np.where(recommendations_["_merge"] == "both", 1, 0)

    n_items = train["JID"].nunique()
    item_popularity_ranks = get_value_popularity_ranks(train["JID"])
    recommendations_["rank_pop"] = recommendations_["JID"].map(item_popularity_ranks)

    recommendations_["proba_user"] = (n_items + 1 - recommendations_["rank"]) / n_items
    recommendations_["proba_any_user"] = (n_items + 1 - recommendations_["rank_pop"]) / n_items

    recommendations_["proba_diff"] = np.maximum(
        recommendations_["proba_user"] - recommendations_["proba_any_user"],
        0.0
    )
    recommendations_["item_serendipity"] = recommendations_["proba_diff"] * recommendations_["is_rel"]
    return recommendations_[["UID", "item_serendipity"]].groupby("UID").agg("mean")


In [ ]:
for model_name, recommendations in recommendations_dict.items():
    serendipity_per_user = calculate_serendipity_per_user(recommendations, train, test)
    print(f"model: {model_name}, mean serendipity: {'{0:.06f}'.format(float(serendipity_per_user.mean()))}\n")

Значение $Serendipity$ для модели с ранжированием по косинусам оказалось выше, что логично из-за того, что для модели, склонной к рекомендациям популярного контента, множитель c разницами вероятностей часто будет равен или очень близок к нулю.

In [ ]:
recommendations_dict['lfm']

In [ ]:
recommendations_dict['lfm_cos']

In [ ]:
recommendations = recommendations_dict['lfm']

recommendations_ = pd.merge(recommendations, test[["UID", "JID"]], how="left", indicator=True)
recommendations_["is_rel"] = np.where(recommendations_["_merge"] == "both", 1, 0)

n_items = train["JID"].nunique()
item_popularity_ranks = get_value_popularity_ranks(train["JID"])
recommendations_["rank_pop"] = recommendations_["JID"].map(item_popularity_ranks)

recommendations_["proba_user"] = (n_items + 1 - recommendations_["rank"]) / n_items
recommendations_["proba_any_user"] = (n_items + 1 - recommendations_["rank_pop"]) / n_items

recommendations_["proba_diff"] = np.maximum(
        recommendations_["proba_user"] - recommendations_["proba_any_user"],
        0.0
    )
recommendations_["item_serendipity"] = recommendations_["proba_diff"] * recommendations_["is_rel"]
rec_grouped =  recommendations_[["UID", "item_serendipity"]].groupby("UID").agg("mean")
rec_grouped

In [ ]:
recommendations = recommendations_dict['lfm_cos']

recommendations_ = pd.merge(recommendations, test[["UID", "JID"]], how="left", indicator=True)
recommendations_["is_rel"] = np.where(recommendations_["_merge"] == "both", 1, 0)

n_items = train["JID"].nunique()
item_popularity_ranks = get_value_popularity_ranks(train["JID"])
recommendations_["rank_pop"] = recommendations_["JID"].map(item_popularity_ranks)

recommendations_["proba_user"] = (n_items + 1 - recommendations_["rank"]) / n_items
recommendations_["proba_any_user"] = (n_items + 1 - recommendations_["rank_pop"]) / n_items

recommendations_["proba_diff"] = np.maximum(
        recommendations_["proba_user"] - recommendations_["proba_any_user"],
        0.0
    )
recommendations_["item_serendipity"] = recommendations_["proba_diff"] * recommendations_["is_rel"]
rec_grouped =  recommendations_[["UID", "item_serendipity"]].groupby("UID").agg("mean")
rec_grouped.mean()

In [ ]:
test

In [ ]:
pd.merge(recommendations, test[["UID", "JID"]], how="left", indicator=True)

In [ ]:
recommendations_dict['random']

In [ ]:
recommendations_dict.keys()

In [ ]:
r = recommendations_dict['random']

In [ ]:
r[r['UID'] == 2]

In [12]:
interactions[interactions['UID'] == 2].sort_values('JID')

,UID,JID,Rating
1282428,2,1,4.08
40663,2,2,-0.29
876204,2,3,6.36
18237,2,4,4.37
30224,2,6,-9.66
...,...,...,...
362683,2,93,-0.29
1396534,2,94,7.86
1056678,2,95,-0.19
1351379,2,97,3.06


In [11]:
interactions[interactions['UID'] == 2]

,UID,JID,Rating
18237,2,4,4.37
23332,2,10,9.22
30224,2,6,-9.66
30415,2,32,-0.92
32211,2,26,7.57
...,...,...,...
1390103,2,8,-5.34
1396534,2,94,7.86
1404094,2,73,8.30
1437961,2,44,8.98


In [2]:
with open('jokes_lfm_model.pkl', 'rb') as f:
    lfm_model = pickle.load(f)

In [ ]:
user_id=1,
model=,
train_matrix=,
user_to_id=,
id_to_item=id_item_mapping,
n_recommendations=n_recommendations,



<24983x100 sparse matrix of type '<class 'numpy.int32'>'
	with 1231109 stored elements in COOrdinate format>

In [8]:
scores = lfm_model.predict(
    user_ids=lfm_dataset._user_id_mapping[2],
    item_ids=np.arange(train_matrix.shape[1]),
    num_threads=20
)

scores

array([-4.0669311e-03, -9.9153787e-01, -1.2242489e+00,  4.1342777e-01,
       -1.1034555e+00,  2.0515226e-02, -7.4589956e-01,  5.2611500e-01,
       -1.7447801e-01, -2.3862697e-01, -1.1688324e+00, -6.1547965e-01,
       -1.4717445e+00, -5.4608756e-01, -6.6251807e-02, -7.4666339e-01,
       -2.6611891e-01, -1.3255584e-01, -9.2180419e-01, -2.7572010e-02,
       -1.1575854e+00,  4.8625398e-01, -3.6753127e-01, -8.9905566e-01,
       -1.5993004e+00, -1.4495698e+00, -3.6515731e-01, -1.3115590e+00,
       -1.0958858e+00, -9.6956140e-01, -4.4142160e-01, -1.2220559e+00,
       -7.4721426e-01, -1.0851709e+00,  5.1495075e-01,  1.1418670e-02,
        2.0496723e-01, -9.7789741e-01, -1.8802877e-01, -1.2111464e+00,
       -7.9641420e-01, -1.2918932e+00, -5.3230591e-02, -4.5810424e-02,
       -1.0161730e+00, -9.5135319e-01, -9.1979094e-02,  4.1357318e-01,
        8.9312769e-02, -1.1230937e+00, -7.7200586e-01, -1.5493228e+00,
       -1.2632064e+00, -1.2078580e+00, -7.9845107e-01, -1.2652503e+00,
      

In [14]:
np.min(scores), np.max(scores)

(-1.5993004, 1.1879255)

In [ ]:



user_watched_items = train_matrix.col[train_matrix.row == user_inner_id]
scores[user_watched_items] = -np.inf

recommended_item_inner_ids = np.argpartition(scores, -np.arange(n_recommendations))[
    -n_recommendations:
][::-1]
recommended_item_ids = [id_to_item[x] for x in recommended_item_inner_ids]
return recommended_item_ids
